In [39]:
import pandas as pd
import numpy as np

from bokeh.io import output_file, output_notebook, show
from bokeh.models import (CDSView, ColumnDataSource, FactorRange, GroupFilter,CustomJSFilter,MultiSelect, BasicTicker, ColorBar, 
                          LinearColorMapper, PrintfTickFormatter,ColumnDataSource, CustomJS, Slider)
from bokeh.layouts import widgetbox, column

from bokeh.plotting import figure
from bokeh.sampledata.unemployment1948 import data
from bokeh.transform import transform
import bokeh.palettes as palettes
from bokeh.layouts import widgetbox, row


In [40]:
CSV_PATH = "Zdhhc_mean_expression_data.csv"
OUTPUT_NAME = "Zdhhc_mean"
HEATMAP_TITLE = "Zdhhc Mean Expression"

In [41]:
Zdhhc_mean = pd.DataFrame(pd.read_csv(CSV_PATH, index_col=0))
genes = list(Zdhhc_mean.index[2:])
brain_regions = list(Zdhhc_mean.loc['Cell Type'].unique())


In [42]:
Zdhhc_mean_update = pd.DataFrame()
for i in range(Zdhhc_mean.shape[1]):
    resilce_data = pd.DataFrame()
    resilce_data['Gene'] = Zdhhc_mean.index[2:]
    resilce_data['Region'] = Zdhhc_mean.iloc[0,i]
    resilce_data['Cell Type'] = Zdhhc_mean.iloc[1,i]
    resilce_data['Expression'] =np.array(Zdhhc_mean.iloc[2:,i])
    Zdhhc_mean_update = pd.concat([Zdhhc_mean_update, resilce_data])
Zdhhc_mean_update['Gene'] = Zdhhc_mean_update['Gene'].astype(str)
Zdhhc_mean_update['Region'] = Zdhhc_mean_update['Region'].astype(str)
Zdhhc_mean_update['Cell Type'] = Zdhhc_mean_update['Cell Type'].astype(str)

Zdhhc_mean_update['Expression'] = Zdhhc_mean_update['Expression'].astype('float')
Zdhhc_mean_update = Zdhhc_mean_update.rename(columns={'Cell Type': 'Cell_Type'})

In [62]:

output_file(OUTPUT_NAME+'.html')




# default subplot to be displayed
subplot = Zdhhc_mean_update[Zdhhc_mean_update['Region']=='microglia']

full_data = ColumnDataSource(Zdhhc_mean_update)
source = ColumnDataSource(subplot)

genes = list(Zdhhc_mean.index[2:])
brain_regions = list(Zdhhc_mean.loc['Cell Type'].unique())
n_color = np.max(np.array(Zdhhc_mean.iloc[2:,:]).ravel())

colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
colors = palettes.plasma(25)
mapper = LinearColorMapper(palette=colors,  
                            low=0,
                            high=subplot['Expression'].max())


x_value = np.unique(source.data['Cell_Type'])

p = figure(width_policy='min',  height_policy='min', title=HEATMAP_TITLE,
           x_range=(FactorRange(factors=x_value)), y_range=(np.unique(source.data['Gene'])),
           toolbar_location=None, tools="", x_axis_location="above",
           
           tooltips = [('Gene', '@Gene'), ('Region','@Region'), ('Cell Type', '@Cell_Type'), ('Expression', '@Expression')])

p.rect(x="Cell_Type", y="Gene", width=1, height=1, source=source,
       line_color=None, fill_color=transform('Expression', mapper))

color_bar = ColorBar(color_mapper=mapper, location=(0, 0),
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     )

p.add_layout(color_bar, 'left')

p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "7px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = 1.0



callback = CustomJS(args ={'fd':full_data, 'source':source, 'p':p, 'cmap':mapper}, code =
            """  
            var r = cb_obj.value;

            var fd_data = fd.data;
 
            var new_cell = [];
            var new_expression = [];
            var new_gene = [];
            var new_region = [];
            var new_index = [];
            var x_vals = [];
            var source_data = source.data;
            
            var regions_x = [];
            var index = 0;
            for (var i = 0; i < (r.length); i++) {
                regions_x.push(r[i]);
                for (var j = 0; j < (fd_data['Region'].length); j++) {
                    if (fd.data['Region'][j] == r[i]) {
                        new_cell.push(fd.data['Cell_Type'][j]);
                        new_expression.push(fd.data['Expression'][j]);
                        new_gene.push(fd.data['Gene'][j]);
                        new_region.push(fd.data['Region'][j]);
                        new_index.push(index);
                        index++;

                }
                }
                }
 
 
            var cell_type_y =source_data.CellType;
            
            

            
            source_data['Cell_Type'] =  new_cell;
            source_data['Expression'] =  new_expression;
            source_data['Gene']   = new_gene;
            source_data['Region'] = new_region;
            source_data['index'] = new_index
            cmap.high = Math.max.apply(Math, new_expression);
            function onlyUnique(value, index, self) { 
            return self.indexOf(value) === index;
            }
            source.change.emit();
            x_vals =  new_cell.filter( onlyUnique );
            p.x_range.factors = x_vals;
            p.change.emit();


           
            """)

gene_callback = CustomJS(args ={'fd':full_data, 'source':source, 'p':p, 'cmap':mapper}, code =
            """  
            var g = cb_obj.value;

            function onlyUnique(value, index, self) { 
            return self.indexOf(value) === index;
            }
            
            
            var cur_reg = source.data['Region'].filter( onlyUnique );
            
            var fd_data = fd.data;
 
            var new_cell = [];
            var new_expression = [];
            var new_gene = [];
            var new_region = [];
            var new_index = [];
            var x_vals = [];
            var source_data = source.data;
            
            var gene_y = [];
            var index = 0;
            for (var i = 0; i < (g.length); i++) {
                gene_y.push(g[i]);
                for (var j = 0; j < (fd_data['Gene'].length); j++) {
                    for(var k=0; k< (cur_reg.length); k++){
                    if (fd.data['Region'][j] == cur_reg[k]){
                    if (fd.data['Gene'][j] == g[i]) {
                        new_cell.push(fd.data['Cell_Type'][j]);
                        new_expression.push(fd.data['Expression'][j]);
                        new_gene.push(fd.data['Gene'][j]);
                        new_region.push(fd.data['Region'][j]);
                        new_index.push(index);
                        index++;
                        }

                }
                }
                }
                }
 
 
            var cell_type_y =source_data.CellType;
            
            

            
            source_data['Cell_Type'] =  new_cell;
            source_data['Expression'] =  new_expression;
            source_data['Gene']   = new_gene;
            source_data['Region'] = new_region;
            source_data['index'] = new_index
            cmap.high = Math.max.apply(Math, new_expression);


            source.change.emit();
            y_vals =  new_gene.filter( onlyUnique );
            x_vals =  new_cell.filter( onlyUnique );
            
            p.y_range.factors = y_vals;
            p.x_range.factors = x_vals;
            
            p.change.emit();

           
            """)









genes_lists = list(Zdhhc_mean_update['Gene'].unique())
active = list( range(len(genes_lists)))

select_genes = MultiSelect(title="Genes:", value=genes_lists,
                           options=genes_lists, callback=gene_callback)

select_regions = MultiSelect(title="Region:", value=["MSN"],
                           options=list(np.unique(full_data.data['Region'])), callback=callback)


data_selector = widgetbox(select_genes, select_regions)
plot = column(data_selector, p)


select_regions.js_on_change('value', callback)
select_regions.js_link('value', select_regions, 'value')

select_genes.js_on_change('value', gene_callback)

show(plot)


In [30]:
display(Zdhhc_mean_update)

,Gene,Region,Cell Type,Expression
0,Zdhhc1,Excitatory cortex/HC,TEGLU1,0.06150
1,Zdhhc2,Excitatory cortex/HC,TEGLU1,0.21000
2,Zdhhc3,Excitatory cortex/HC,TEGLU1,0.28700
3,Zdhhc4,Excitatory cortex/HC,TEGLU1,0.04490
4,Zdhhc5,Excitatory cortex/HC,TEGLU1,0.06420
...,...,...,...,...
18,Zdhhc20,microglia,MGL1,0.14200
19,Zdhhc21,microglia,MGL1,0.01160
20,Zdhhc22,microglia,MGL1,0.00000
21,Zdhhc23,microglia,MGL1,0.00000


<bound method Model.layout of Figure(id='2946', ...)>